In [50]:
from pyspark.sql import SparkSession
from pyspark import SparkContext

In [38]:
from pyspark.sql.functions import sum, col

Условие: создайте csv файл с таким содержимым:

title,author,genre,sales,year
"1984", "George Orwell", "Science Fiction", 5000, 1949
"The Lord of the Rings", "J.R.R. Tolkien", "Fantasy", 3000, 1954
"To Kill a Mockingbird", "Harper Lee", "Southern Gothic", 4000, 1960
"The Catcher in the Rye", "J.D. Salinger", "Novel", 2000, 1951
"The Great Gatsby", "F. Scott Fitzgerald", "Novel", 4500, 1925

Задание:

— Используя Spark прочитайте данные из файла csv.
— Фильтруйте данные, чтобы оставить только книги, продажи которых превышают 3000 экземпляров.
— Сгруппируйте данные по жанру и вычислите общий объем продаж для каждого жанра.
— Отсортируйте данные по общему объему продаж в порядке убывания.
— Выведите результаты на экран.

In [51]:
spark = SparkSession.builder.master('local[*]').appName('work_with_csv_file').getOrCreate()

In [53]:
schema = ["title","author","genre","sales","year"]
data = [("1984", "George Orwell", "Science Fiction", 5000, 1949),
        ("The Lord of the Rings", "J.R.R. Tolkien", "Fantasy", 3000, 1954),
        ("To Kill a Mockingbird", "Harper Lee", "Southern Gothic", 4000, 1960),
        ("The Catcher in the Rye", "J.D. Salinger", "Novel", 2000, 1951),
        ("The Great Gatsby", "F. Scott Fitzgerald", "Novel", 4500, 1925)]
books_df = spark.createDataFrame(data, schema)

In [28]:
books_df.show()

+--------------------+-------------------+---------------+-----+----+
|               title|             author|          genre|sales|year|
+--------------------+-------------------+---------------+-----+----+
|                1984|      George Orwell|Science Fiction| 5000|1949|
|The Lord of the R...|     J.R.R. Tolkien|        Fantasy| 3000|1954|
|To Kill a Mocking...|         Harper Lee|Southern Gothic| 4000|1960|
|The Catcher in th...|      J.D. Salinger|          Novel| 2000|1951|
|    The Great Gatsby|F. Scott Fitzgerald|          Novel| 4500|1925|
+--------------------+-------------------+---------------+-----+----+



In [55]:
books_df.write.option("header",True).csv('books.csv')

In [56]:
books_df_read = spark.read.csv('books.csv', sep=',', inferSchema=True, header=True)

In [57]:
books_df_read.show()

+--------------------+-------------------+---------------+-----+----+
|               title|             author|          genre|sales|year|
+--------------------+-------------------+---------------+-----+----+
|To Kill a Mocking...|         Harper Lee|Southern Gothic| 4000|1960|
|The Catcher in th...|      J.D. Salinger|          Novel| 2000|1951|
|    The Great Gatsby|F. Scott Fitzgerald|          Novel| 4500|1925|
|                1984|      George Orwell|Science Fiction| 5000|1949|
|The Lord of the R...|     J.R.R. Tolkien|        Fantasy| 3000|1954|
+--------------------+-------------------+---------------+-----+----+



Задание:

— Используя Spark прочитайте данные из файла csv.
— Фильтруйте данные, чтобы оставить только книги, продажи которых превышают 3000 экземпляров.
— Сгруппируйте данные по жанру и вычислите общий объем продаж для каждого жанра.
— Отсортируйте данные по общему объему продаж в порядке убывания.
— Выведите результаты на экран.

In [58]:
df2 = books_df_read.filter(col("sales") > 3000)
df2.show()

+--------------------+-------------------+---------------+-----+----+
|               title|             author|          genre|sales|year|
+--------------------+-------------------+---------------+-----+----+
|To Kill a Mocking...|         Harper Lee|Southern Gothic| 4000|1960|
|    The Great Gatsby|F. Scott Fitzgerald|          Novel| 4500|1925|
|                1984|      George Orwell|Science Fiction| 5000|1949|
+--------------------+-------------------+---------------+-----+----+



In [47]:
df3 = books_df_read.groupBy("genre").agg(sum("sales")).orderBy(col("sum(sales)").desc())
df3.show()

+---------------+----------+
|          genre|sum(sales)|
+---------------+----------+
|          Novel|      6500|
|Science Fiction|      5000|
|Southern Gothic|      4000|
|        Fantasy|      3000|
+---------------+----------+



In [60]:
spark.stop()